## Prepare

In [2]:
from neuroconv.datainterfaces import PhySortingInterface, CsvTimeIntervalsInterface
from neuroconv import NWBConverter
from neuroconv.utils.dict import load_dict_from_file, dict_deep_update
from pathlib import Path
from datetime import datetime,timezone
from zoneinfo import ZoneInfo

from pynwb import NWBHDF5IO, NWBFile
from pynwb.ecephys import LFP, ElectricalSeries
import matplotlib.pyplot as plt
import phylib
from phylib.io.model import load_model
import numpy as np
import pandas as pd



## make colormap

In [2]:


# colorcycler = (cycler(color=["darkorange", "gold", "lawngreen", "seagreen", "cornflowerblue", 
#           "mediumorchid",  "mediumvioletred", "crimson", "saddlebrown","darkgoldenrod", 
#           "olivedrab", "seagreen", "darkslategray", "deepskyblue","maroon"]))
# #map1 = LinearSegmentedColormap.from_list("mycmap", colors)

#, "saddlebrown"
#"maroon", "saddlebrown"

colorlist=["orangered", "darkorange", "gold", "lawngreen", "seagreen", "cornflowerblue", 
           "deepskyblue", "darkblue", "mediumorchid", "indigo", "mediumvioletred", 
           "crimson"]


## Extract unit and trial data 

In [3]:
filename = 'EMR22009_20230323'

with NWBHDF5IO(f"C:\\Users\\Ellen\\OneDrive - University of Massachusetts\\20220622_PersonalComp\\NPX\\NWBFiles\\{filename}.nwb", "r") as io:
    read_nwbfile = io.read()
    
    #print(read_nwbfile.trials)
    units=read_nwbfile.units
    units_df=units.to_dataframe()
    units_df=units_df[units_df['quality']!='noise']
    #goodunits_df=units_df[units_df['quality']=='good']
    trials=read_nwbfile.trials
    trials_df=trials.to_dataframe()
    
io.close()


In [4]:
units_df.head()

,spike_times,unit_name,amp,original_cluster_id,sh,n_spikes,KSLabel,depth,ch,quality,ContamPct,fr,Amplitude,waveform_shape,p2t_time,p2t_dist
id,,,,,,,,,,,,,,,,
0,"[0.12276666666666666, 0.3393333333333333, 0.55...",12,93.629334,12.0,0.0,10727.0,mua,7380.0,10.0,good,29.4,3.122623,907.2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.200000,93.629333
1,"[20.1984, 51.2817, 51.58906666666667, 51.94156...",14,143.688965,14.0,0.0,78723.0,good,7380.0,11.0,good,0.0,22.916214,3053.5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.166667,143.688965
3,"[2.040233333333333, 3.2939666666666665, 6.0238...",20,134.820572,20.0,0.0,58448.0,good,7340.0,14.0,good,2.5,17.014175,2318.9,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.166667,134.820572
4,"[0.1852, 0.3183666666666667, 0.373433333333333...",21,146.746918,21.0,0.0,17000.0,mua,7340.0,15.0,good,33.9,4.948689,3784.7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.166667,146.746918
5,"[0.37293333333333334, 0.7979, 2.3471, 3.103433...",23,121.027695,23.0,0.0,20578.0,good,7340.0,15.0,good,4.6,5.990242,1044.9,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.133333,121.027695


## add location data to file - need to check parameters for each file manually first

In [6]:
#make location data - a list that we will use to add column to NWB file 
#need to manually identify parameters for where is LC - for this file it is based on depth 

locationdata=[]

for i in range(len(units_df)):
    if units_df.iloc[i].quality == 'noise':
        locationdata.append('nan')    
    elif units_df.iloc[i].depth < 6900:
        locationdata.append('cerebellum')
    elif units_df.iloc[i].depth > 7300:
        locationdata.append('brainstem')
    else:
        locationdata.append('lc')




In [ ]:

filename = 'EMR22009_20230323'
with NWBHDF5IO(f"C:\\Users\\Ellen\\OneDrive - University of Massachusetts\\20220622_PersonalComp\\NPX\\NWBFiles\\{filename}.nwb", "r+") as io:
    read_nwbfile = io.read()
    
    read_nwbfile.units.add_column(name='location', description='brain region of recording site', data=locationdata)

    units=read_nwbfile.units
    units_df=units.to_dataframe()
    # units_df=units_df[units_df['quality']!='noise']
    # goodunits_df=units_df[units_df['quality']=='good']

    io.write(read_nwbfile)
    io.close()
    #print(read_nwbfile.acquisition)
    units_df.head()

In [ ]:
potential_lc=units_df[units_df['location']=='lc']
potential_cerebellum=units_df[units_df['location']=='cerebellum']
potential_brainstem=units_df[units_df['location']=='brainstem']

## ZSCORE FUNCTIONS

In [4]:
# GET ZSCORE OF LC NEURONS AREOUND CUE, LP, LEVEROUT
import numpy as np
import pandas as pd

def bin_whole_file(data, neuron, binsize):
    temp = data.iloc[neuron]
    temp2=temp['spike_times']
    wholefile_bin = np.histogram(temp2, bins=int(5000*(1000/binsize)), range=(0,5000))
    return wholefile_bin[0]
    

def bin_around_event(cue_times, spikes, seconds, binsize):
    col=((2*seconds)*(1000/binsize))
    temp_allevents_oneneuron=np.zeros((1,int(col)))

    for event in cue_times:
        hold= np.histogram(spikes, bins=int((2*seconds)*(1000/binsize)), range=((event-seconds),(event+seconds)))
        hold=hold[0]
        hold=np.atleast_2d(hold)
        temp_allevents_oneneuron= np.vstack((temp_allevents_oneneuron,(hold)))

    temp_allevents_oneneuron=np.delete(temp_allevents_oneneuron,0,0)
    allevets_oneneuron=temp_allevents_oneneuron.mean(axis=0)

    return(allevets_oneneuron)



def zscore_around_event(binned_data, binsize, neuron, data):
    ytemp=binned_data
    xtemp=range(len(binned_data))

    wholetemp=bin_whole_file(data, neuron, binsize)
    z_mean=np.mean(wholetemp)
    z_sd=np.std(wholetemp)

    z_zscore = []
    for i in ytemp:
        z_zscore.append((i-z_mean)/z_sd)
        #print(z_zscore)
    
    return z_zscore

# row=potential_lc.iloc[0]
# spikes=row['spike_times']
# averleverout=trials_df[trials_df['cue']==2]
# avercue_times=np.array(avercue['start_time'])
# seconds=1
# binsize=100
# neuron=0

# neuron_one=bin_around_event(avercue_times,spikes,seconds, binsize)
# zscore_neuronone = zscore_around_event(neuron_one,binsize, neuron)

seconds=1
binsize=100


zscore_lc_leverpress_aversive=[]
bin_lc_leverpress_aversive=[]
cueevent=trials_df[trials_df['cue']==2]
cueevent=cueevent[cueevent['response']==1]
event_times=np.array(cueevent['leverpress_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_leverpress_aversive.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_leverpress_aversive.append(zscore_around_event(bin_lc_leverpress_aversive[i],binsize, neuron, potential_lc))



zscore_lc_leverpress_appetative=[]
bin_lc_leverpress_appetative=[]
cueevent=trials_df[trials_df['cue']==1]
cueevent=cueevent[cueevent['response']==1]
event_times=np.array(cueevent['leverpress_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_leverpress_appetative.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_leverpress_appetative.append(zscore_around_event(bin_lc_leverpress_appetative[i],binsize, neuron,potential_lc))




zscore_lc_leverpress_neutral=[]
bin_lc_leverpress_neutral=[]
cueevent=trials_df[trials_df['cue']==3]

cueevent=cueevent[cueevent['response']!=0]
event_times=np.array(cueevent['leverpress_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_leverpress_neutral.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_leverpress_neutral.append(zscore_around_event(bin_lc_leverpress_neutral[i],binsize, neuron,potential_lc))

###CUE##############

zscore_lc_cue_aversive=[]
bin_lc_cue_aversive=[]
cueevent=trials_df[trials_df['cue']==2]
#cueevent=cueevent[cueevent['response']==1]
event_times=np.array(cueevent['start_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_cue_aversive.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_cue_aversive.append(zscore_around_event(bin_lc_cue_aversive[i],binsize, neuron, potential_lc))



zscore_lc_cue_appetative=[]
bin_lc_cue_appetative=[]
cueevent=trials_df[trials_df['cue']==1]
#cueevent=cueevent[cueevent['response']==1]
event_times=np.array(cueevent['start_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_cue_appetative.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_cue_appetative.append(zscore_around_event(bin_lc_cue_appetative[i],binsize, neuron,potential_lc))




zscore_lc_cue_neutral=[]
bin_lc_cue_neutral=[]
cueevent=trials_df[trials_df['cue']==3]
#cueevent=cueevent[cueevent['response']!=0]
event_times=np.array(cueevent['start_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_cue_neutral.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_cue_neutral.append(zscore_around_event(bin_lc_cue_neutral[i],binsize, neuron,potential_lc))



######### LEVEROUT #################

zscore_lc_leverout_aversive=[]
bin_lc_leverout_aversive=[]
cueevent=trials_df[trials_df['cue']==2]
#cueevent=cueevent[cueevent['response']==1]
event_times=np.array(cueevent['leverout_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_leverout_aversive.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_leverout_aversive.append(zscore_around_event(bin_lc_leverout_aversive[i],binsize, neuron, potential_lc))



zscore_lc_leverout_appetative=[]
bin_lc_leverout_appetative=[]
cueevent=trials_df[trials_df['cue']==1]
#cueevent=cueevent[cueevent['response']==1]
event_times=np.array(cueevent['leverout_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_leverout_appetative.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_leverout_appetative.append(zscore_around_event(bin_lc_leverout_appetative[i],binsize, neuron,potential_lc))




zscore_lc_leverout_neutral=[]
bin_lc_leverout_neutral=[]
cueevent=trials_df[trials_df['cue']==3]
#cueevent=cueevent[cueevent['response']!=0]
event_times=np.array(cueevent['leverout_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_leverout_neutral.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_leverout_neutral.append(zscore_around_event(bin_lc_leverout_neutral[i],binsize, neuron,potential_lc))




KeyError: 'cue'

In [ ]:
# PLOT ZSCORE AROUND CUE ONSET 

import matplotlib.pyplot as plt
import numpy as np
xaxis=np.arange(-seconds, seconds, (binsize/1000) )

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,5))

i=0
for neuron in zscore_lc_cue_aversive:
    ax1.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax1.legend(potential_lc['unit_name'], loc='upper left')
ax1.set_title('Aversive Cue')
ax1.set_ylabel('z score')
ax1.set_xlabel('Time from cue onset (s)')
ax1.set_ylim(-.5,2)

i=0
for neuron in zscore_lc_cue_appetative:
    ax2.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax2.legend(potential_lc['unit_name'], loc='upper left')
ax2.set_title('Appetative Cue')
ax2.set_ylabel('z score')
ax2.set_xlabel('Time from cue onset (s)')
ax2.set_ylim(-.5,2)

i=0
for neuron in zscore_lc_cue_neutral:
    ax3.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax3.legend(potential_lc['unit_name'], loc='upper left')
ax3.set_title('Neutral Cue')
ax3.set_ylabel('z score')
ax3.set_xlabel('Time from cue onset (s)')
ax3.set_ylim(-.5,2)


In [ ]:
#PLOT RAW BIN AORUND CUE ONSET
import matplotlib.pyplot as plt
import numpy as np
xaxis=np.arange(-seconds, seconds, (binsize/1000) )

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,5))
i=0
for neuron in bin_lc_cue_aversive:
    ax1.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax1.legend(potential_lc['unit_name'], loc='upper left')
ax1.set_title('Aversive Cue')
ax1.set_ylabel('bin')
ax1.set_xlabel('Time from cue onset (s)')

i=0
for neuron in bin_lc_cue_appetative:
    ax2.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax2.legend(potential_lc['unit_name'], loc='upper left')
ax2.set_title('Appetative Cue')
ax2.set_ylabel('avg bin')
ax2.set_xlabel('Time from cue onset (s)')

i=0
for neuron in bin_lc_cue_neutral:
    ax3.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax3.legend(potential_lc['unit_name'], loc='upper left')
ax3.set_title('Neutral Cue')
ax3.set_ylabel('avg bin')
ax3.set_xlabel('Time from cue onset (s)')


In [ ]:
# PLOT ZSCORE AROUND LEVER OUT
import matplotlib.pyplot as plt
import numpy as np
xaxis=np.arange(-seconds, seconds, (binsize/1000) )

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,5))

i=0
for neuron in zscore_lc_leverout_aversive:
    ax1.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax1.legend(potential_lc['unit_name'], loc='upper left')
ax1.set_title('Aversive Cue')
ax1.set_ylabel('z score')
ax1.set_xlabel('Time from lever out (s)')
ax1.set_ylim(-.5,2)

i=0
for neuron in zscore_lc_leverout_appetative:
    ax2.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax2.legend(potential_lc['unit_name'], loc='upper left')
ax2.set_title('Appetative Cue')
ax2.set_ylabel('z score')
ax2.set_xlabel('Time from lever out (s)')
ax2.set_ylim(-.5,2)

i=0
for neuron in zscore_lc_leverout_neutral:
    ax3.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax3.legend(potential_lc['unit_name'], loc='upper left')
ax3.set_title('Neutral Cue')
ax3.set_ylabel('z score')
ax3.set_xlabel('Time from lever out (s)')
ax3.set_ylim(-.5,2)


In [ ]:
#PLOT ZSCORE AROUND LEVERPRESS ON CORRECT TRIALS ONLY
import matplotlib.pyplot as plt
import numpy as np
xaxis=np.arange(-seconds, seconds, (binsize/1000) )

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,5))
i=0
for neuron in zscore_lc_leverpress_aversive:
    ax1.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax1.legend(potential_lc['unit_name'], loc='upper left')
ax1.set_title('Aversive Cue')
ax1.set_ylabel('z score')
ax1.set_xlabel('Time from lever press (s)')
ax1.set_ylim(-.5,2)

i=0
for neuron in zscore_lc_leverpress_appetative:
    ax2.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax2.legend(potential_lc['unit_name'], loc='upper left')
ax2.set_title('Appetative Cue')
ax2.set_ylabel('z score')
ax2.set_xlabel('Time from lever press (s)')
ax2.set_ylim(-.5,2)

i=0
for neuron in zscore_lc_leverpress_neutral:
    ax3.plot(xaxis,neuron, c=colorlist[i])
    i+=1
ax3.legend(potential_lc['unit_name'], loc='upper left')
ax3.set_title('Neutral Cue')
ax3.set_ylabel('z score')
ax3.set_xlabel('Time from lever press (s)')
ax3.set_ylim(-.5,2)


## ZSCORE  for anes recording

In [5]:
#FOR ANES - GET ZSCORE OF LC NEURONS AREOUND JAW AND FOOTPINCH
import numpy as np
import pandas as pd

def bin_whole_file(data, neuron, binsize):
    temp = data.iloc[neuron]
    temp2=temp['spike_times']
    wholefile_bin = np.histogram(temp2, bins=int(5000*(1000/binsize)), range=(0,5000))
    return wholefile_bin[0]
    

def bin_around_event(cue_times, spikes, seconds, binsize):
    col=((2*seconds)*(1000/binsize))
    temp_allevents_oneneuron=np.zeros((1,int(col)))

    for event in cue_times:
        hold= np.histogram(spikes, bins=int((2*seconds)*(1000/binsize)), range=((event-seconds),(event+seconds)))
        hold=hold[0]
        hold=np.atleast_2d(hold)
        temp_allevents_oneneuron= np.vstack((temp_allevents_oneneuron,(hold)))

    temp_allevents_oneneuron=np.delete(temp_allevents_oneneuron,0,0)
    allevets_oneneuron=temp_allevents_oneneuron.mean(axis=0)

    return(allevets_oneneuron)



def zscore_around_event(binned_data, binsize, neuron, data):
    ytemp=binned_data
    xtemp=range(len(binned_data))

    wholetemp=bin_whole_file(data, neuron, binsize)
    z_mean=np.mean(wholetemp)
    z_sd=np.std(wholetemp)

    z_zscore = []
    for i in ytemp:
        z_zscore.append((i-z_mean)/z_sd)
        #print(z_zscore)
    
    return z_zscore



seconds=2
binsize=100


zscore_lc_contrafootpinch=[]
bin_lc_contrafootpinch=[]
cueevent=trials_df[trials_df['footpinch']==1]
event_times=np.array(cueevent['start_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_contrafootpinch.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_contrafootpinch.append(zscore_around_event(bin_lc_contrafootpinch[i],binsize, neuron, potential_lc))



zscore_lc_ipsifootpinch=[]
bin_lc_ipsifootpinch=[]
cueevent=trials_df[trials_df['footpinch']==2]
event_times=np.array(cueevent['start_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_ipsifootpinch.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_ipsifootpinch.append(zscore_around_event(bin_lc_ipsifootpinch[i],binsize, neuron,potential_lc))




zscore_lc_contrafootpinch_post=[]
bin_lc_contrafootpinch_post=[]
cueevent=trials_df[trials_df['footpinch']==3]
event_times=np.array(cueevent['start_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_contrafootpinch_post.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_contrafootpinch_post.append(zscore_around_event(bin_lc_contrafootpinch_post[i],binsize, neuron, potential_lc))



zscore_lc_ipsifootpinch_post=[]
bin_lc_ipsifootpinch_post=[]
cueevent=trials_df[trials_df['footpinch']==4]
event_times=np.array(cueevent['start_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_ipsifootpinch_post.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_ipsifootpinch_post.append(zscore_around_event(bin_lc_ipsifootpinch_post[i],binsize, neuron,potential_lc))

### jaw ##############

zscore_lc_jaw=[]
bin_lc_jaw=[]
cueevent=trials_df[trials_df['jaw']==1]
event_times=np.array(cueevent['start_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_jaw.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_jaw.append(zscore_around_event(bin_lc_jaw[i],binsize, neuron, potential_lc))

zscore_lc_jaw_post=[]
bin_lc_jaw_post=[]
cueevent=trials_df[trials_df['jaw']==2]
event_times=np.array(cueevent['start_time'])

for i in range(len(potential_lc)):
    neuron=i
    row=potential_lc.iloc[i]
    spikes=row['spike_times']
    bin_lc_jaw_post.append(bin_around_event(event_times,spikes,seconds, binsize))
    zscore_lc_jaw_post.append(zscore_around_event(bin_lc_jaw_post[i],binsize, neuron, potential_lc))




NameError: name 'potential_lc' is not defined

In [6]:
for i in range(len(zscore_lc_contrafootpinch)):
    row=zscore_lc_contrafootpinch[i]
    


average_score_lc_contrafootpinch[0]
temp=(zscore_lc_contrafootpinch[0])
(temp[15:25])

NameError: name 'average_score_lc_contrafootpinch' is not defined

In [ ]:
# PLOT ZSCORE AROUND footpinch

import matplotlib.pyplot as plt
import numpy as np
xaxis=np.arange(-seconds, seconds, (binsize/1000) )

fig, ax = plt.subplots(2,2, figsize=(25,25))
#ax1.legend(potential_lc['unit_name'], loc='upper left')

for neuron in zscore_lc_contrafootpinch:
    ax[0,0].plot(xaxis,neuron) #, c=colorlist[i])
ax[0,0].set_title('Contra FP')
ax[0,0].set_ylabel('z score')

for neuron in zscore_lc_ipsifootpinch:
    ax[0,1].plot(xaxis,neuron) #, c=colorlist[i])
ax[0,1].set_title('Ipsi FP')
ax[0,1].set_ylabel('z score')


for neuron in zscore_lc_contrafootpinch_post:
    ax[1,0].plot(xaxis,neuron) #, c=colorlist[i])
ax[1,0].set_title('Contra FP post clon')
ax[1,0].set_ylabel('z score')


for neuron in zscore_lc_ipsifootpinch_post:
    ax[1,1].plot(xaxis,neuron) #, c=colorlist[i])
ax[1,1].set_title('Ipsi FP post clon')
ax[1,1].set_ylabel('z score')


In [ ]:
# all potential LC around contra FP - 5 per figure
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(3,3, figsize=(25,25))


legend=[]
for neuron in range(0,5):
    ax[0,0].plot(xaxis,zscore_lc_contrafootpinch[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[0,0].legend(legend, loc='upper left')
ax[0,0].set_ylim(-1,8)


legend=[]
for neuron in range(5,10):
    ax[0,1].plot(xaxis,zscore_lc_contrafootpinch[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[0,1].legend(legend, loc='upper left')
ax[0,1].set_ylim(-1,8)

legend=[]
for neuron in range(10,15):
    ax[0,2].plot(xaxis,zscore_lc_contrafootpinch[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[0,2].legend(legend, loc='upper left')
ax[0,2].set_ylim(-1,8)

legend=[]
for neuron in range(15,20):
    ax[1,0].plot(xaxis,zscore_lc_contrafootpinch[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[1,0].legend(legend, loc='upper left')
ax[1,0].set_ylim(-1,8)

legend=[]
for neuron in range(20,25):
    ax[1,1].plot(xaxis,zscore_lc_contrafootpinch[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[1,1].legend(legend, loc='upper left')
ax[1,1].set_ylim(-1,8)

legend=[]
for neuron in range(25,30):
    ax[1,2].plot(xaxis,zscore_lc_contrafootpinch[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[1,2].legend(legend, loc='upper left')
ax[1,2].set_ylim(-1,8)

legend=[]
for neuron in range(30,35):
    ax[2,0].plot(xaxis,zscore_lc_contrafootpinch[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[2,0].legend(legend, loc='upper left')
ax[2,0].set_ylim(-1,8)

legend=[]
for neuron in range(35,40):
    ax[2,1].plot(xaxis,zscore_lc_contrafootpinch[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[2,1].legend(legend, loc='upper left')
ax[2,1].set_ylim(-1,8)

legend=[]
for neuron in range(40,43):
    ax[2,2].plot(xaxis,zscore_lc_contrafootpinch[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[2,2].legend(legend, loc='upper left')
ax[2,2].set_ylim(-1,8)


In [ ]:
# all potential LC around ipsi FP - 5 per figure
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(3,3, figsize=(25,25))


legend=[]
for neuron in range(0,5):
    ax[0,0].plot(xaxis,zscore_lc_ipsifootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[0,0].legend(legend, loc='upper left')
ax[0,0].set_ylim(-1,8)


legend=[]
for neuron in range(5,10):
    ax[0,1].plot(xaxis,zscore_lc_ipsifootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[0,1].legend(legend, loc='upper left')
ax[0,1].set_ylim(-1,8)

legend=[]
for neuron in range(10,15):
    ax[0,2].plot(xaxis,zscore_lc_ipsifootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[0,2].legend(legend, loc='upper left')
ax[0,2].set_ylim(-1,8)

legend=[]
for neuron in range(15,20):
    ax[1,0].plot(xaxis,zscore_lc_ipsifootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[1,0].legend(legend, loc='upper left')
ax[1,0].set_ylim(-1,8)

legend=[]
for neuron in range(20,25):
    ax[1,1].plot(xaxis,zscore_lc_ipsifootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[1,1].legend(legend, loc='upper left')
ax[1,1].set_ylim(-1,8)

legend=[]
for neuron in range(25,30):
    ax[1,2].plot(xaxis,zscore_lc_ipsifootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[1,2].legend(legend, loc='upper left')
ax[1,2].set_ylim(-1,8)

legend=[]
for neuron in range(30,35):
    ax[2,0].plot(xaxis,zscore_lc_ipsifootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[2,0].legend(legend, loc='upper left')
ax[2,0].set_ylim(-1,8)

legend=[]
for neuron in range(35,40):
    ax[2,1].plot(xaxis,zscore_lc_ipsifootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[2,1].legend(legend, loc='upper left')
ax[2,1].set_ylim(-1,8)

legend=[]
for neuron in range(40,43):
    ax[2,2].plot(xaxis,zscore_lc_ipsifootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[2,2].legend(legend, loc='upper left')
ax[2,2].set_ylim(-1,8)


In [ ]:
# all potential LC around contra FP post clon - 5 per figure
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(3,3, figsize=(25,25))


legend=[]
for neuron in range(0,5):
    ax[0,0].plot(xaxis,zscore_lc_contrafootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[0,0].legend(legend, loc='upper left')
ax[0,0].set_ylim(-1,8)


legend=[]
for neuron in range(5,10):
    ax[0,1].plot(xaxis,zscore_lc_contrafootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[0,1].legend(legend, loc='upper left')
ax[0,1].set_ylim(-1,8)

legend=[]
for neuron in range(10,15):
    ax[0,2].plot(xaxis,zscore_lc_contrafootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[0,2].legend(legend, loc='upper left')
ax[0,2].set_ylim(-1,8)

legend=[]
for neuron in range(15,20):
    ax[1,0].plot(xaxis,zscore_lc_contrafootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[1,0].legend(legend, loc='upper left')
ax[1,0].set_ylim(-1,8)

legend=[]
for neuron in range(20,25):
    ax[1,1].plot(xaxis,zscore_lc_contrafootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[1,1].legend(legend, loc='upper left')
ax[1,1].set_ylim(-1,8)

legend=[]
for neuron in range(25,30):
    ax[1,2].plot(xaxis,zscore_lc_contrafootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[1,2].legend(legend, loc='upper left')
ax[1,2].set_ylim(-1,8)

legend=[]
for neuron in range(30,35):
    ax[2,0].plot(xaxis,zscore_lc_contrafootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[2,0].legend(legend, loc='upper left')
ax[2,0].set_ylim(-1,8)

legend=[]
for neuron in range(35,40):
    ax[2,1].plot(xaxis,zscore_lc_contrafootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[2,1].legend(legend, loc='upper left')
ax[2,1].set_ylim(-1,8)

legend=[]
for neuron in range(40,43):
    ax[2,2].plot(xaxis,zscore_lc_contrafootpinch_post[neuron])
    legend.append(potential_lc['unit_name'].iloc[neuron])
ax[2,2].legend(legend, loc='upper left')
ax[2,2].set_ylim(-1,8)


## extra trim lc from anes recording


In [ ]:
from itertools import compress

lc_unitnames=['60','73','851','1261','1255','1257','1265','1275','1288','1303','1307','1744','1292']

temp=potential_lc['unit_name'].isin(lc_unitnames)
trim_lc=potential_lc[temp]


trim_zscore_lc_contrafootpinch=list(compress(zscore_lc_contrafootpinch,temp))
trim_zscore_lc_ipsifootpinch=list(compress(zscore_lc_ipsifootpinch,temp))
trim_zscore_lc_contrafootpinch_post=list(compress(zscore_lc_contrafootpinch_post,temp))
trim_zscore_lc_ipsifootpinch_post=list(compress(zscore_lc_ipsifootpinch_post,temp))

trim_zscore_lc_jaw=list(compress(zscore_lc_jaw,temp))
trim_zscore_lc_jaw_post=list(compress(zscore_lc_jaw_post,temp))



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(4,3, figsize=(25,25))


legend=[]
for neuron in range(0,5):
    ax[0,0].plot(xaxis,trim_zscore_lc_contrafootpinch[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[0,0].legend(legend, loc='upper left')
ax[0,0].set_ylim(-1,8)


legend=[]
for neuron in range(5,10):
    ax[0,1].plot(xaxis,trim_zscore_lc_contrafootpinch[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[0,1].legend(legend, loc='upper left')
ax[0,1].set_ylim(-1,8)

legend=[]
for neuron in range(10,13):
    ax[0,2].plot(xaxis,trim_zscore_lc_contrafootpinch[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[0,2].legend(legend, loc='upper left')
ax[0,2].set_ylim(-1,8)

###
legend=[]
for neuron in range(0,5):
    ax[1,0].plot(xaxis,trim_zscore_lc_contrafootpinch_post[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[1,0].legend(legend, loc='upper left')
ax[1,0].set_ylim(-1,8)


legend=[]
for neuron in range(5,10):
    ax[1,1].plot(xaxis,trim_zscore_lc_contrafootpinch_post[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[1,1].legend(legend, loc='upper left')
ax[1,1].set_ylim(-1,8)

legend=[]
for neuron in range(10,13):
    ax[1,2].plot(xaxis,trim_zscore_lc_contrafootpinch_post[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[1,2].legend(legend, loc='upper left')
ax[1,2].set_ylim(-1,8)


###
legend=[]
for neuron in range(0,5):
    ax[2,0].plot(xaxis,trim_zscore_lc_ipsifootpinch[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[2,0].legend(legend, loc='upper left')
ax[2,0].set_ylim(-1,8)


legend=[]
for neuron in range(5,10):
    ax[2,1].plot(xaxis,trim_zscore_lc_ipsifootpinch[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[2,1].legend(legend, loc='upper left')
ax[2,1].set_ylim(-1,8)

legend=[]
for neuron in range(10,13):
    ax[2,2].plot(xaxis,trim_zscore_lc_ipsifootpinch[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[2,2].legend(legend, loc='upper left')
ax[2,2].set_ylim(-1,8)



###
legend=[]
for neuron in range(0,5):
    ax[3,0].plot(xaxis,trim_zscore_lc_ipsifootpinch_post[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[3,0].legend(legend, loc='upper left')
ax[3,0].set_ylim(-1,8)


legend=[]
for neuron in range(5,10):
    ax[3,1].plot(xaxis,trim_zscore_lc_ipsifootpinch_post[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[3,1].legend(legend, loc='upper left')
ax[3,1].set_ylim(-1,8)

legend=[]
for neuron in range(10,13):
    ax[3,2].plot(xaxis,trim_zscore_lc_ipsifootpinch_post[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[3,2].legend(legend, loc='upper left')
ax[3,2].set_ylim(-1,8)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(2,3, figsize=(15,15))


legend=[]
for neuron in range(0,5):
    ax[0,0].plot(xaxis,trim_zscore_lc_jaw[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[0,0].legend(legend, loc='upper left')
ax[0,0].set_ylim(-1,15)


legend=[]
for neuron in range(5,10):
    ax[0,1].plot(xaxis,trim_zscore_lc_jaw[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[0,1].legend(legend, loc='upper left')
ax[0,1].set_ylim(-1,8)

legend=[]
for neuron in range(10,13):
    ax[0,2].plot(xaxis,trim_zscore_lc_jaw[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[0,2].legend(legend, loc='upper left')
ax[0,2].set_ylim(-1,8)

###
legend=[]
for neuron in range(0,5):
    ax[1,0].plot(xaxis,trim_zscore_lc_jaw_post[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[1,0].legend(legend, loc='upper left')
ax[1,0].set_ylim(-1,15)


legend=[]
for neuron in range(5,10):
    ax[1,1].plot(xaxis,trim_zscore_lc_jaw_post[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[1,1].legend(legend, loc='upper left')
ax[1,1].set_ylim(-1,8)

legend=[]
for neuron in range(10,13):
    ax[1,2].plot(xaxis,trim_zscore_lc_jaw_post[neuron])
    legend.append(trim_lc['unit_name'].iloc[neuron])
ax[1,2].legend(legend, loc='upper left')
ax[1,2].set_ylim(-1,8)


## raster 

In [ ]:

def calc_raster(neuron):
    raster=[]
    for event in range(len(trials_df['start_time'])):
        eventtime=trials_df['leverout_time'].iloc[event]
        tempneuron=trim_lc['spike_times'].iloc[neuron]
        tempneuron=tempneuron-eventtime
        tempraster=tempneuron[tempneuron>-1]
        tempraster=tempraster[tempraster<1]
        raster.append(tempraster)
    return raster 




In [ ]:
colors1=["seagreen", "seagreen", "deepskyblue", "red","deepskyblue", "red","deepskyblue","deepskyblue","deepskyblue",
         "black","seagreen","seagreen","deepskyblue","red","deepskyblue","red", "deepskyblue" ]




#plt.eventplot(raster, colors=colors1)

In [ ]:

colorcuelist=[]
for cue in trials_df['cue']:
    if cue == 1:
        colorcuelist.append('seagreen')
    elif cue == 2:
        colorcuelist.append('red')
    elif cue == 3:
        colorcuelist.append('black')



colorresplist=[]
for response in trials_df['response']:
    if response == 1:
        colorresplist.append('seagreen')
    elif response == 3:
        colorresplist.append('red')
    elif response == 2:
        colorresplist.append('blue')
    elif response == 0:
        colorresplist.append('black')
    else:
        colorresplist.append('purple')



In [ ]:

# raster OF LC ISI - ONE NEURON/PLOT
import matplotlib.pyplot as plt
import numpy as np


nrows, ncols= 2,5
fig, axs = plt.subplots(nrows,ncols, figsize=(25,10), sharex=True, sharey=True)

for i, ax in enumerate(axs.flatten()):

    raster=calc_raster(i)
    ax.eventplot(raster, colors=colorresplist)
    ax.axvline(x = 0, color = 'black')

    # ax.hist(isidata[i], bins=50, range=(0,.1), 
    #                   density=True, label=trim_lc['unit_name'].iloc[i], color=colorlist[i])
    # ax.set_title(f'Unit {trim_lc['unit_name'].iloc[i]}')

plt.tight_layout
plt.show()

# axs[0,0].legend()
#plt.hist(isidata, bins=10, range=(0,.1), histtype='bar', density=True)
#plt.show() 

## ISI

In [ ]:
# FIND AND INSERT ISI DATA FOR EACH UNIT TO POTENTIAL LC 
def find_isi(data, neuron):
    isi=[]
    tempspikes=data.iloc[neuron].spike_times
    for x in range(len(tempspikes)-2):
        isi.append(tempspikes[x+1]-tempspikes[x])
    return np.array(isi)

#temp=find_isi(potential_lc,0)

isidata=[]

for i in range(len(trim_lc)):
    temp=find_isi(trim_lc,i)
    isidata.append(temp)

trim_lc["isi"]=isidata


In [ ]:
# HISTOGRAM OF ALL POTENTIAL LC NEURONS - ISI
import matplotlib.pyplot as plt
import numpy as np


#used to delete rows that are not LC - prob not the best way to do this
#isidatatrim=isidata
#isidataoutlierthree=isidatatrim.pop(3)
#isidataoutlierone=isidatatrim.pop(1)



plt.hist(isidata, bins=10, range=(0,.5), histtype='bar', density=True, color=colorlist)
plt.show() 

In [ ]:
# HISTOGRAM OF LC ISI - ONE NEURON/PLOT
import matplotlib.pyplot as plt
import numpy as np


nrows, ncols= 2,5
fig, axs = plt.subplots(nrows,ncols, figsize=(25,10), sharex=False, sharey=False)

for i, ax in enumerate(axs.flatten()):
    ax.hist(isidata[i], bins=50, range=(0,.1), 
                      density=True, label=trim_lc['unit_name'].iloc[i], color=colorlist[i])
    ax.set_title(f'Unit {trim_lc['unit_name'].iloc[i]}')

plt.tight_layout
plt.show()

# axs[0,0].legend()
#plt.hist(isidata, bins=10, range=(0,.1), histtype='bar', density=True)
#plt.show() 

In [ ]:
## RETURN DENSITY PLOT ATTEMPT - scatter for all units

import matplotlib.pyplot as plt
import numpy as np

nrows, ncols= 2,5
fig, axs = plt.subplots(nrows,ncols, figsize=(30,10), sharex=False, sharey=False)

for i, ax in enumerate(axs.flatten()):
    temp=isidata[i]
    temp_x=temp[0:-1]
    temp_y=temp[1:]
    temp_x_log=np.log(temp_x)
    temp_y_log=np.log(temp_y)

    ax.scatter(temp_x_log, temp_y_log, s=3, c=colorlist[i])
    ax.set_xlabel('log(ISI n)')
    ax.set_ylabel('log(ISI n+1)')
    ax.set_title(f'Unit {trim_lc['unit_name'].iloc[i]}')

    

plt.tight_layout
plt.show()



In [ ]:
#make a return density and histogram for one unit 

def scatter_hist(x, y, ax, ax_histx, ax_histy,i):
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # the scatter plot:
    ax.scatter(x, y, s=2, c=colorlist[i])

    # now determine nice limits by hand:
    binwidth = 0.25
    xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
    lim = (int(xymax/binwidth) + 1) * binwidth

    bins = np.arange(-lim, lim + binwidth, binwidth)
    ax_histx.hist(x, bins=bins, color=colorlist[i])
    ax_histy.hist(y, bins=bins, orientation='horizontal', color=colorlist[i])




# Create a Figure, which doesn't have to be square.
fig = plt.figure(layout='constrained')
# Create the main Axes, leaving 25% of the figure space at the top and on the
# right to position marginals.
ax = fig.add_gridspec(top=0.75, right=0.75).subplots()
# The main Axes' aspect can be fixed.
ax.set(aspect=1)
# Create marginal Axes, which have 25% of the size of the main Axes.  Note that
# the inset Axes are positioned *outside* (on the right and the top) of the
# main Axes, by specifying axes coordinates greater than 1.  Axes coordinates
# less than 0 would likewise specify positions on the left and the bottom of
# the main Axes.
ax_histx = ax.inset_axes([0, 1.05, 1, 0.25], sharex=ax)
ax_histy = ax.inset_axes([1.05, 0, 0.25, 1], sharey=ax)
# Draw the scatter plot and marginals.
#scatter_hist(x, y, ax, ax_histx, ax_histy)



i=8

temp=isidata[i]
temp_x=temp[0:-1]
temp_y=temp[1:]
temp_x_log=np.log(temp_x)
temp_y_log=np.log(temp_y)

scatter_hist(temp_x_log, temp_y_log, ax, ax_histx, ax_histy,i)
plt.show()


## Plot unit data basics 

In [ ]:
# PLOT AMPLITUDE, FR, DEPTH SCATTERPLOTS

import matplotlib.pyplot as plt
import numpy as np

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,5))
ax1.plot(units_df['fr'],(-units_df['depth']),'o')
ax1.set_title('Unit FR x Depth')
ax1.set_xlabel('Firing Rate')
ax1.set_ylabel('Depth')

ax2.plot(units_df['Amplitude'],(-units_df['depth']),'o')
ax2.set_title('Unit Amplitude x Depth')
ax2.set_xlabel('Amplitude')
ax2.set_ylabel('Depth')


ax3.plot(units_df['Amplitude'],(units_df['fr']),'o')
ax3.set_title('Unit Amplitude x FR')
ax3.set_xlabel('Amplitude')
ax3.set_ylabel('Firing Rate')




plt.show()

In [ ]:
#PLOT DEPTH, FR, AMPLITUDE SCATTER FOR LC NEURONS

import matplotlib.pyplot as plt
import numpy as np

fig, (ax1, ax2, ax3,ax4) = plt.subplots(1,4, figsize=(25,5))
ax1.plot(trim_lc['fr'],(-trim_lc['depth']),'o')
ax1.set_title('Unit FR x Depth')
ax1.set_xlabel('Firing Rate')
ax1.set_ylabel('Depth')

ax2.plot(trim_lc['Amplitude'],(-trim_lc['depth']),'o')
ax2.set_title('Unit Amplitude x Depth')
ax2.set_xlabel('Amplitude')
ax2.set_ylabel('Depth')


ax3.plot(trim_lc['Amplitude'],(trim_lc['fr']),'o')
ax3.set_title('Unit Amplitude x FR')
ax3.set_xlabel('Amplitude')
ax3.set_ylabel('Firing Rate')

ax4.plot(trim_lc['fr'],(trim_lc['ch']),'o')
ax4.set_title('Unit FR x Channel')
ax4.set_xlabel('Firing Rate')
ax4.set_ylabel('Channel')




plt.show()

In [ ]:
trim_lc=trim_lc[trim_lc['fr']<10]

In [ ]:
#PLOT DEPTH, FR, AMPLITUDE SCATTER FOR ABOVE, LC, BELOW NEURONS
# 
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(3,3, figsize=(15,15))


ax[0,0].plot(potential_cerebellum['fr'],(-potential_cerebellum['depth']),'o')
ax[0,0].set_title('Unit FR x Depth')
ax[0,0].set_xlabel('Firing Rate')
ax[0,0].set_ylabel('Depth')

ax[0,1].plot(potential_cerebellum['Amplitude'],(-potential_cerebellum['depth']),'o')
ax[0,1].set_title('Unit Amplitude x Depth')
ax[0,1].set_xlabel('Amplitude')
ax[0,1].set_ylabel('Depth')

ax[0,2].plot(potential_cerebellum['Amplitude'],(potential_cerebellum['fr']),'o')
ax[0,2].set_title('Unit Amplitude x FR')
ax[0,2].set_xlabel('Amplitude')
ax[0,2].set_ylabel('Firing Rate')



ax[1,0].plot(potential_lc['fr'],(-potential_lc['depth']),'o')
ax[1,0].set_xlabel('Firing Rate')
ax[1,0].set_ylabel('Depth')

ax[1,1].plot(potential_lc['Amplitude'],(-potential_lc['depth']),'o')
ax[1,1].set_xlabel('Amplitude')
ax[1,1].set_ylabel('Depth')

ax[1,2].plot(potential_lc['Amplitude'],(potential_lc['fr']),'o')
ax[1,2].set_xlabel('Amplitude')
ax[1,2].set_ylabel('Firing Rate')



ax[2,0].plot(potential_brainstem['fr'],(-potential_brainstem['depth']),'o')
ax[2,0].set_xlabel('Firing Rate')
ax[2,0].set_ylabel('Depth')

ax[2,1].plot(potential_brainstem['Amplitude'],(-potential_brainstem['depth']),'o')
ax[2,1].set_xlabel('Amplitude')
ax[2,1].set_ylabel('Depth')

ax[2,2].plot(potential_brainstem['Amplitude'],(potential_brainstem['fr']),'o')
ax[2,2].set_xlabel('Amplitude')
ax[2,2].set_ylabel('Firing Rate')




plt.show()

## NWB widget panel

In [ ]:
from nwbwidgets.panel import Panel

Panel()

In [ ]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
filename = 'EMR22009_20230323'

io = NWBHDF5IO(f"C:\\Users\\Ellen\\OneDrive - University of Massachusetts\\20220622_PersonalComp\\NPX\\NWBFiles\\{filename}.nwb", "r")
nwb = io.read()

#nwb2widget(nwb)

## SCRAPS

In [ ]:
row=len(avercue_times)
col=(2*(5000*25))
temp_allevents_oneneuron=np.zeros((row,col))

check=(np.histogram(spikes, bins=2*(5000*25), range=((event-(5000*25)),(event+(5000*25)))))
check=check[0]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 


avercue=trials_df[trials_df['cue']==2]
avercue_times=np.array(avercue['start_time'])

row=potential_lc.iloc[0]
spikes=row['spike_times']


#for index, row in potential_lc.iterrows():     
#spikes=row['spike_times']


ytemp=neuron_one
xtemp=range(2000*50)

#grab cooresponding whole bin value for mean and std
wholetemp=(wholefile_25mssbin[0])
ytemp_whole=wholetemp[0]

z_mean=np.mean(ytemp_whole)
z_sd=np.std(ytemp_whole)


z_zscore = []
for i in ytemp:
    z_zscore.append((i-z_mean)/z_sd)
print(z_zscore)


plt.plot(xtemp,z_zscore, linewidth=1)


In [ ]:
#this will plot the histogram of spikes over the recording session for one unit
#not really useful big pictue but sanity check 

import matplotlib.pyplot as plt
import numpy as np


arr=units_df.spike_times[14]
plt.hist(arr[1:], bins=np.linspace(0,5000,500))

plt.show

In [ ]:

from nwbwidgets.allen import load_allen_widgets
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
#pip install allensdk
import allensdk
#import allensdk.brain_observatory.ecephys.nwb

filename = 'EMR22009_20230327'
fpath = f"C:\\Users\\Ellen\\OneDrive - University of Massachusetts\\20220622_PersonalComp\\NPX\\NWBFiles\\{filename}.nwb"
nwb=NWBHDF5IO(fpath,'r').read()
